# Notebook about the research done on retrieving data for word2vec embeddings


## Trying to find related words using twitter tweet search api

## Define all base terms for which we want to search tweets for that may come in member profiles

In [3]:
# terms=["Programmer","Coding","Javascript","Cloud Computing","Manager","Finance","Design","Art","Dribbble","Github","Python","Data","Science","Commerce","Machine Learning",\
#       "Engineer","IBM","Chartered Accountant","Figma","Adobe","Music","Audio","Language","Vocabulary","Designer","Coder","Mathematician","Creative","Logic","Logical"]
# terms specially listed for teams app 
terms=["Programmer","Coding","Deep Learning","Cloud Computing","Javascript","Backend","Frontend","Data Science","Dribbble","Github","Machine Learning",\
       "Engineer","Figma","Sketch","Blender","Mathematician","Audio Programming","ReactJS","VueJS","CSS","Design Engineer","Operating System",\
       "Linux","Scripting","Shell Script","Leader"]

In [4]:
terms

['Programmer',
 'Coding',
 'Deep Learning',
 'Cloud Computing',
 'Javascript',
 'Backend',
 'Frontend',
 'Data Science',
 'Dribbble',
 'Github',
 'Machine Learning',
 'Engineer',
 'Figma',
 'Sketch',
 'Blender',
 'Mathematician',
 'Audio Programming',
 'ReactJS',
 'VueJS',
 'CSS',
 'Design Engineer',
 'Operating System',
 'Linux',
 'Scripting',
 'Shell Script',
 'Leader']

### Twitter API sample code

```python
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    query = "from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    tweet_fields = "tweet.fields=author_id"
    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}".format(
        query, tweet_fields
    )
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()
```

In [5]:
import requests
import os
import json

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def create_url(query):
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
#     tweet_fields = "tweet.fields=author_id"
    #max_results=100
    url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10".format(
        query
    )
    return url

In [6]:
def auth():
    return "AAAAAAAAAAAAAAAAAAAAAC6vMwEAAAAA5uumFGxBM1e4Kfyfr8E5TJseEZw%3DHV6mjkDp1QrpVm1bpi2yZrvAEkpqhxCK91fGkjl5gevPwtLScM"

```python
# fetch tweet data initial
for i in terms:
    bearer_token = auth()
    url = create_url(i)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    with open("./word_dataset/"+i+'_data.json', 'a') as f:
        json.dump(json_response["data"], f)
```

```python
# fetch more tweet data
def create_url(query,next_token):
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
#     tweet_fields = "tweet.fields=author_id"
    #max_results=100
    if next_token:
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&next_token={}".format(
            query,next_token
        )
    else:
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100".format(
            query
        )
    return url

next_token=[i for i in terms]
iters=5
for j in range(iters):
    k=0
    for i in terms:
        bearer_token = auth()
        if(j!=0):
            url = create_url(i,next_token[k])
        else:
            url = create_url(i,None)
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(url, headers)
        if(json_response["meta"]["next_token"]):
            next_token[k]=json_response["meta"]["next_token"]
        k+=1
        with open("./word_dataset/"+i+"_data.json", "r+") as file:
            data = json.load(file)
            data.extend(json_response["data"])
            file.seek(0)
            json.dump(data, file)
```

## Start cleaning data then extract keywords using rake or spacy

```python
import re
for i in terms:
    with open("./word_dataset/"+i+"_data.json", "r+") as file:
            data = json.load(file)
            arr=[]
            for k in data:
                # add clean up line here and saved cleaned data somewhere
                final_cleaned = str(re.sub(r"[^a-zA-Z0-9]+", ' ', k["text"]))
                arr.append(final_cleaned)
            file2=open("./word_dataset/"+i+"_cleaned.json","a+")
            json.dump(arr,file2)
```

In [8]:
# str(re.sub(r"[^a-zA-Z0-9]+", ' ', k["text"]))

## Start removing stop words and useless words from sentences 

In [9]:
# !conda install -c conda-forge rake_nltk -y


In [22]:
# !python -c "import nltk; nltk.download('stopwords')"
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{'couldn', "doesn't", 'with', 'being', 'not', 'nor', 'most', 'yourselves', 'too', 'off', 'whom', 'at', 'some', 'been', 'because', 'were', "weren't", 'you', "should've", 'if', 'this', 'between', 'him', 'ourselves', 'now', "didn't", 'her', 'o', 'yourself', "aren't", 'we', 'to', 've', 'from', 'than', 'doesn', 'has', 'she', "shouldn't", 'won', 'is', 'y', 'himself', 'own', 'why', 'myself', 'further', 'did', "won't", "hasn't", 'there', 'shan', 'be', 'isn', 'theirs', 'doing', 'themselves', 'll', 'that', 'having', 'or', 'should', 'once', 'against', 'mustn', 'what', 'have', 'didn', "mightn't", "isn't", 'a', 'any', 'aren', 'm', 'do', "she's", 'ma', 'few', 'below', 'haven', 'of', 'shouldn', 'down', 'for', 'mightn', 'will', 'their', 't', 'when', "wouldn't", 'only', 'before', 'don', 'my', 'itself', 'by', 'was', "you've", 'd', 'ain', 'which', "wasn't", 'same', 'above', 'an', 'wouldn', 'again', 'yours', 'hadn', 'am', "hadn't", 'me', 'but', 'then', 'no', "don't", "shan't", 'does', 'such', 'who', 'hers

## Removed stop words from tweets lets build a pandas dataframe with text

```python
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
for i in terms:
    with open("./cleaned_dataset/"+i+"_cleaned.json", "r+") as file:
        data = json.load(file)
        arr=[]
        for k in data:
            # add clean up line here and saved cleaned data somewhere
            final_cleaned = ' '.join([word for word in k.split() if word not in stop_words])
            arr.append(final_cleaned)
            file2=open("./cleaned_dataset/"+i+"_stopworded.json","w+")
            json.dump(arr,file2)
```        

In [29]:
import pandas as pd

In [30]:
all_text_df=pd.DataFrame()

In [31]:
text_db=[]
for i in terms:
     with open("./cleaned_dataset/"+i+"_stopworded.json", "r+") as file:
        data = json.load(file)
        for k in data:
            text_db.append(k)

In [32]:
len(text_db)

10455

In [33]:
all_text_df["text"]=text_db

In [35]:
all_text_df

,text
0,gekido SykooSam I must born programmer
1,RT PR0GRAMMERHUM0R Every programmer like https...
2,RT carrotpharma Join welcoming Pharma Consulta...
3,RT motherofnodejs Never programming coding pro...
4,RT motherofnodejs Never programming coding pro...
...,...
10450,RT toryboypierce Former SNP leader Alec Salmon...
10451,RT Fereeha PPP leader TalpurTaimur said snake ...
10452,RT nowthisnews HAPPENING NOW SCHUMER SPEAKS TO...
10453,RT waller73 Just year leader done LiverpoolMayor


In [36]:
all_text_df.to_csv("tweet_text_db.csv")

In [37]:
# !conda install -c anaconda gensim -y

Solving environment: done

## Package Plan ##

  environment location: /home/kaypee/miniconda3/envs/ds

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boto-2.49.0                |           py38_0         1.6 MB  anaconda
    boto3-1.17.12              |     pyhd3eb1b0_0          70 KB
    botocore-1.20.12           |     pyhd3eb1b0_1         3.5 MB
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py38_0         160 KB  anaconda
    gensim-3.8.3               |   py38h2531618_2        18.5 MB
    google-api-core-1.22.2     |           py38_0         101 KB  anaconda
    google-cloud-core-1.4.3    |             py_0          29 KB  anaconda
    google-cloud-storage-1.32.0|             py_0          64 KB  anaconda
    google-crc32c-1.0.0        |   py38h7b6447c_0          26 KB  a

In [38]:
import nltk

In [40]:
all_text_df['tokenized'] = all_text_df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

In [41]:
all_text_df

,text,tokenized
0,gekido SykooSam I must born programmer,"[gekido, SykooSam, I, must, born, programmer]"
1,RT PR0GRAMMERHUM0R Every programmer like https...,"[RT, PR0GRAMMERHUM0R, Every, programmer, like,..."
2,RT carrotpharma Join welcoming Pharma Consulta...,"[RT, carrotpharma, Join, welcoming, Pharma, Co..."
3,RT motherofnodejs Never programming coding pro...,"[RT, motherofnodejs, Never, programming, codin..."
4,RT motherofnodejs Never programming coding pro...,"[RT, motherofnodejs, Never, programming, codin..."
...,...,...
10450,RT toryboypierce Former SNP leader Alec Salmon...,"[RT, toryboypierce, Former, SNP, leader, Alec,..."
10451,RT Fereeha PPP leader TalpurTaimur said snake ...,"[RT, Fereeha, PPP, leader, TalpurTaimur, said,..."
10452,RT nowthisnews HAPPENING NOW SCHUMER SPEAKS TO...,"[RT, nowthisnews, HAPPENING, NOW, SCHUMER, SPE..."
10453,RT waller73 Just year leader done LiverpoolMayor,"[RT, waller73, Just, year, leader, done, Liver..."
